In [8]:
import numpy as np
from sklearn.neighbors import radius_neighbors_graph
import chainer
import chainer.functions as F
import glob
import struct

In [9]:
DATA_PATH = '/home/evan/Data/nbody_simulations/N_{0}/DM*/{1}_dm.z=0{2}000'
#=============================================================================
# Data utils
#=============================================================================
def read_sim(file_list, n_P):
    """ reads simulation data from disk and returns

    Args:
        file_list: (list<str>) paths to files
        n_P: (int) number of particles base (n_P**3 particles)
    """
    num_particles = n_P**3
    dataset = []
    for file_name in file_list:
        this_set = []
        with open(file_name, "rb") as f:
            for i in range(num_particles*6):
                s = struct.unpack('=f',f.read(4))
                this_set.append(s[0])
        dataset.append(this_set)
    dataset = np.array(dataset).reshape([len(file_list),num_particles,6])  
    return dataset

def load_datum(n_P, redshift, normalize_data=False):
    """ loads two redshift datasets from proper data directory

    Args:
        redshift: (float) redshift
        n_P: (int) base of number of particles (n_P**3 particles)
    """
    N_P = 10000 if n_P == 32 else 1000
    glob_paths = glob.glob(DATA_PATH.format(N_P, 'xv', redshift))
    X = read_sim(glob_paths, n_P)
    if normalize_data:
        X = normalize(X)
    return X

def normalize(X_in):
    """ Normalize features
    coordinates are rescaled to (0,1)
    velocities normalized by mean/std    
    """
    x_r = np.reshape(X_in, [-1,6])
    coo, vel = np.split(x_r, [3], axis=-1)
    coo_min = np.min(coo, axis=0)
    coo_max = np.max(coo, axis=0)
    #coo_mean, coo_std = np.mean(coo,axis=0), np.std(coo,axis=0)
    #x_r[:,:3] = (x_r[:,:3] - coo_mean) / (coo_std)
    vel_mean = np.mean(vel, axis=0)
    vel_std  = np.std( vel, axis=0)
    x_r[:,:3] = (x_r[:,:3] - coo_min) / (coo_max - coo_min)
    x_r[:,3:] = (x_r[:,3:] - vel_mean) / vel_std
    X_out = np.reshape(x_r,X_in.shape).astype(np.float32) # just convert to float32 here
    return X_out

In [10]:
num_particles = 16
zX = 0.6
X = load_datum(num_particles, zX, normalize_data=True)


In [15]:
mb_size = 10
N = X.shape[0]
idx = np.random.choice(N, mb_size)
x = X[idx]

In [16]:
'''
Density stuff
'''
rad = 0.01
graph = np.zeros([mb_size, N, N], dtype=np.float32)

In [17]:
graph = radius_neighbors_graph(x[0,:,:3], rad, include_self=True)

In [19]:
type(graph)

scipy.sparse.csr.csr_matrix